In [50]:
import pandas as pd

In [51]:
airbnb = pd.read_csv("../data/clean/inside_airbnb_clean.gz")
airbnb.head()

,id,last_scraped,host_id,host_name,neighbourhood,latitude,longitude,room_type,bedrooms,availability_365,calculated_host_listings_count,geom,zipcode,borough,year,availability_pct
0,2539,2022-09-07,2787,John,Kensington,40.64529,-73.97238,Private room,1.0,356,9,"40.64529,-73.97238",11218,BROOKLYN,2022,97.534247
1,5121,2022-09-07,7356,Garon,Bedford-Stuyvesant,40.68535,-73.95512,Private room,1.0,335,2,"40.68535,-73.95512",11238,BROOKLYN,2022,91.780822
2,45910,2022-09-07,204539,Mark,Ridgewood,40.70309,-73.89963,Entire home/apt,5.0,365,6,"40.70309,-73.89963",11385,QUEENS,2022,100.000000
3,5136,2022-09-07,7378,Rebecca,Sunset Park,40.66265,-73.99454,Entire home/apt,2.0,179,1,"40.66265,-73.99454",11215,BROOKLYN,2022,49.041096
4,45935,2022-09-07,204586,L,Mott Haven,40.80635,-73.92201,Private room,1.0,83,1,"40.80635,-73.92201",10454,BRONX,2022,22.739726


In [52]:
# Group by year and borough and count the number of unique ids
# Returns the yearly count for each borough
airbnb.groupby(["year", "borough", "zipcode"])["id"].count().to_frame().reset_index()

,year,borough,zipcode,id
0,2015,BRONX,10451,29
1,2015,BRONX,10452,42
2,2015,BRONX,10453,61
3,2015,BRONX,10454,69
4,2015,BRONX,10455,38
...,...,...,...,...
1424,2022,STATEN_ISLAND,10308,18
1425,2022,STATEN_ISLAND,10309,6
1426,2022,STATEN_ISLAND,10310,43
1427,2022,STATEN_ISLAND,10312,11


In [53]:
rental = pd.read_csv("../data/clean/nyc_rental_prices.csv")
rental.head()

,rent,borough,zipcode,year
0,16000.0,MANHATTAN,10013,2022
1,8250.0,MANHATTAN,10012,2022
2,2200.0,MANHATTAN,10031,2022
3,13000.0,MANHATTAN,10011,2022
4,55000.0,MANHATTAN,10065,2022


In [54]:
rental.groupby(["year", "borough", "zipcode"])["rent"].count().to_frame().reset_index()

,year,borough,zipcode,rent
0,2015,BRONX,10457,10
1,2015,BRONX,10458,10
2,2015,BRONX,10461,10
3,2015,BRONX,10462,10
4,2015,BRONX,10463,10
...,...,...,...,...
849,2022,STATEN_ISLAND,10306,11
850,2022,STATEN_ISLAND,10308,2
851,2022,STATEN_ISLAND,10309,2
852,2022,STATEN_ISLAND,10312,12


In [55]:
home = pd.read_csv("../data/clean/zhome_value.csv")
home.head()

,zipcode,state,city,borough,year,home_value
0,11368,NY,New York,QUEENS,2002,249885.0
1,11385,NY,New York,QUEENS,2002,333404.0
2,11236,NY,New York,BROOKLYN,2002,255687.0
3,11208,NY,New York,BROOKLYN,2002,235676.0
4,10467,NY,New York,BRONX,2002,174696.0


In [56]:
home.groupby(["year", "borough", "zipcode"])["home_value"].count().to_frame().reset_index()

,year,borough,zipcode,home_value
0,2002,BRONX,10452,11
1,2002,BRONX,10453,12
2,2002,BRONX,10454,12
3,2002,BRONX,10456,12
4,2002,BRONX,10457,12
...,...,...,...,...
3628,2022,STATEN_ISLAND,10308,10
3629,2022,STATEN_ISLAND,10309,10
3630,2022,STATEN_ISLAND,10310,10
3631,2022,STATEN_ISLAND,10312,10
